In [18]:
import numpy as np
import opt_einsum as oe

nao = 241

test = oe.contract_path(
    "ijkl,i,j,kl->",
    (nao, nao, nao, nao),
    (nao,),
    (nao,),
    (nao, nao),
    optimize="optimal",
    memory_limit="max_input",
)

print(test)

ValueError: Einstein sum subscript 'ijkl' does not contain the correct number of indices for operand 0.

In [3]:
11 // 2

5